In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from PIL import Image
import numpy as np
import glob
                                                                                                                                                                                
import logging
from Pegasus.api import *
logging.basicConfig(level=logging.DEBUG)

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from utils.wf import split_data_filenames

DEBUG:matplotlib:(private) matplotlib data path: /usr/local/lib64/python3.6/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:matplotlib data path: /usr/local/lib64/python3.6/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/scitech/.config/matplotlib
DEBUG:matplotlib:matplotlib version 3.3.3
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keywor

DEBUG:matplotlib:CACHEDIR=/home/scitech/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/scitech/.cache/matplotlib/fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [4]:
#! sudo python3 -m pip install --upgrade pip
#! sudo python3 -m pip install --upgrade Pillow numpy torchvision matplotlib

In [2]:
imagesList = glob.glob('Images2/*.png')
annotationList = glob.glob('annotations2/*.xml')

In [3]:
train_filenames,val_filenames,test_filenames, files_split_dict = split_data_filenames(imagesList)

In [4]:
def create_ann_list(filenames):
    files_ids = []
    imgs = []

    for filename in filenames:
        stringList = filename.split(".")
        tempName = stringList[0]
        digit = tempName[len(tempName)-1]
        files_ids.append(digit)
        fname = filename.split("/")[-1]
        imgs.append(File(fname))
    ann = []
    for ids in files_ids:
        name = 'maksssksksss{}.xml'.format(ids)
        ann.append(File(name))
    return imgs, ann

In [5]:
def create_resized_filelist(filenames):
    files_ids = []

    for filename in filenames:
        stringList = filename.split(".")
        tempName = stringList[0]
        digit = tempName[len(tempName)-1]
        files_ids.append(digit)
    resized_files = []
    for ids in files_ids:
        name = 'resized_maksssksksss{}.png'.format(ids)
        resized_files.append(File(name))
    return resized_files

In [6]:
train_imgs, train_ann = create_ann_list(train_filenames)
val_imgs, val_ann = create_ann_list(val_filenames)
test_imgs, test_ann = create_ann_list(test_filenames)

In [7]:
train_resized_files = create_resized_filelist(train_filenames)
val_resized_files = create_resized_filelist(val_filenames)
test_resized_files = create_resized_filelist(test_filenames)

In [8]:
rc = ReplicaCatalog()

inputFiles = []
for img in imagesList:
    fileName = img.split("/")[1]
    img_file = File(fileName)
    inputFiles.append(img_file)
    rc.add_replica("local", img_file, "/home/scitech/shared-data/face-recognition-wf/"+str(img))

In [9]:
annFiles = []

for ann in annotationList:
    fileName = ann.split("/")[1]
    ann_file = File(fileName)
    annFiles.append(ann_file)
    rc.add_replica("local", ann_file, "/home/scitech/shared-data/face-recognition-wf/"+str(ann))
rc.write()

In [10]:
image_resize = Transformation(
                "image_resize",
                site="local",
                pfn="/home/scitech/shared-data/face-recognition-wf/img_resize.py",
                is_stageable=True
            )

In [11]:
tc = TransformationCatalog()
tc.add_transformations(image_resize)
tc.write()

In [12]:
wf = Workflow("mask_detection_workflow")


In [13]:
job_image_resize_train = Job(image_resize)
job_image_resize_train.add_inputs(*train_imgs, *train_ann)
job_image_resize_train.add_outputs(*train_resized_files)

In [14]:
job_image_resize_val = Job(image_resize)
job_image_resize_val.add_inputs(*val_imgs, *val_ann)
job_image_resize_val.add_outputs(*val_resized_files)

In [15]:
job_image_resize_test = Job(image_resize)
job_image_resize_test.add_inputs(*test_imgs, *test_ann)
job_image_resize_test.add_outputs(*test_resized_files)

In [16]:
# add jobs to workflow
wf.add_jobs(
    job_image_resize_train,
    job_image_resize_val,
    job_image_resize_test
)

# run workflow
try:
    wf.plan(submit=True)
    wf.wait()
    wf.statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2021.01.24 20:12:59.621 UTC:
2021.01.24 20:12:59.633 UTC:   -----------------------------------------------------------------------
2021.01.24 20:12:59.646 UTC:   File for submitting this DAG to HTCondor           : mask_detection_workflow-0.dag.condor.sub
2021.01.24 20:12:59.663 UTC:   Log of DAGMan debugging messages                 : mask_detection_workflow-0.dag.dagman.out
2021.01.24 20:12:5

[##############------------------------------------]  27.3% ..Failure (Completed: 3, Queued: 0, Running: 3, Failed: 0)



######################
# pegasus-statistics #
######################
pegasus-monitord still running. Please wait for it to complete.




pegasus-monitord still running. Please wait for it to complete.

